### Import libraries and dataset

In [ ]:
# Import libraries
from sklearn.datasets import load_boston
import numpy as np
import matplotlib.pyplot as plt
import warnings
from sklearn.model_selection import KFold
from sklearn.preprocessing import PolynomialFeatures

## Avoid printing out warnings
with warnings.catch_warnings():
     warnings.filterwarnings("ignore")
     X, y = load_boston(return_X_y=True)


In [ ]:
boston = load_boston()
print(boston_dataset.keys())

### Closed form solution

In [ ]:
X = boston.data
y = boston.target

# normalize dataset
u = np.mean(X, axis=0)
std = np.std(X, axis=0)
X = (X - u)/std

# append the column of 1s
one = np.ones((X.shape[0], 1))
X = np.hstack((one, X))

In [ ]:
# define the number of folds for cross-validation
k = 10
kf = KFold(n_splits=k, shuffle=True, random_state=42)
mse_values = []

In [ ]:
# perform k-fold cross-validation
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    # (X.TX)-1 X.Ty
    a = np.linalg.inv(np.dot(X_train.T, X_train))
    b = np.dot(X_train.T, y_train)
    theta = np.dot(a, b)

    y_prediction = np.dot(X_test, theta)
    
    mse_fold = np.mean((y_test - y_prediction) ** 2)
    mse_values.append(mse_fold)

In [ ]:
average_mse = np.mean(mse_values)
print("Average MSE across", k, "folds:", average_mse)

### Ridge regression

In [ ]:
X = boston.data
y = boston.target

# normalize dataset
u = np.mean(X, axis=0)
std = np.std(X, axis=0)
X = (X - u)/std

# append the column of 1s
one = np.ones((X.shape[0], 1))
X = np.hstack((one, X))

In [ ]:
# define the number of folds for cross-validation
k = 10
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# define the range of lambda values using np.logspace
lambda_values = np.logspace(1, 7, num=13)

# initialize a dictionary to store the mse scores for each lambda value
mse_values_dict = {}

In [ ]:
# perform k-fold cross-validation for each lambda value
for lambda_param in lambda_values:
    mse_values = []

    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # (X.TX+λI)-1.X.Ty
        XTX = np.dot(X_train.T, X_train)
        I = np.eye(XTX.shape[0])
        theta = np.dot(np.linalg.inv(XTX + lambda_param * I), np.dot(X_train.T, y_train))

        # calculate MSE for this fold
        y_prediction = np.dot(X_test, theta)
        mse_fold = np.mean((y_test - y_prediction) ** 2)
        mse_values.append(mse_fold)

    # store the MSE values for this lambda value
    mse_values_dict[lambda_param] = np.mean(mse_values)

In [ ]:
# find the lambda value with the lowest average mse
best_lambda = min(mse_values_dict, key=mse_values_dict.get)
best_mse = mse_values_dict[best_lambda]

print(f'Best lambda: 10^{int(np.log10(best_lambda))}')
print(f'Corresponding average MSE: {best_mse}')

### Evaluating model performance

In [ ]:
X = boston.data
y = boston.target

# normalize dataset
u = np.mean(X, axis=0)
std = np.std(X, axis=0)
X = (X - u)/std

# append the column of 1s
one = np.ones((X.shape[0], 1))
X = np.hstack((one, X))

In [ ]:
# define the number of folds for cross-validation
k = 10
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# define the range of lambda values using np.logspace
lambda_values = np.logspace(1, 7, num=13)

# initialize a list to store the MSE scores for best lambda value
mse_scores = []

In [ ]:
# perform k-fold cross-validation for best lambda value
for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        # (X.TX+λI)-1.X.Ty
        XTX = np.dot(X_train.T, X_train)
        I = np.eye(XTX.shape[0])
        theta = np.dot(np.linalg.inv(XTX + best_lambda * I), np.dot(X_train.T, y_train))

        y_prediction_train = np.dot(X_train, theta)
        y_prediction_test = np.dot(X_test, theta)

        mse_train = np.mean((y_train - y_prediction_train) ** 2)
        mse_test = np.mean((y_test - y_prediction_test) ** 2)

        mse_scores.append({'train': mse_train, 'test': mse_test})

average_mse_train = np.mean([fold['train'] for fold in mse_scores])
average_mse_test = np.mean([fold['test'] for fold in mse_scores])

In [ ]:
print(f'average mse on training set: {average_mse_train}')
print(f'average mse on test set: {average_mse_test}')

### Polynomial transformation and ridge regression

In [ ]:
X = boston.data
y = boston.target

# normalize dataset
u = np.mean(X, axis=0)
std = np.std(X, axis=0)
X = (X - u) / std

# create polynomial features of degree 2
poly = PolynomialFeatures(degree=2)
X_poly = poly.fit_transform(X)

# append the column of 1s
one = np.ones((X_poly.shape[0], 1))
X_poly = np.hstack((one, X_poly))

# define the number of folds (e.g., 5)
k = 10
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# define the range of lambda values using np.logspace
lambda_values = np.logspace(1, 7, num=13)

# initialize a dictionary to store the mse scores for each lambda value
mse_scores_dict = {}

# perform k-fold cross-validation for each lambda value
for lambda_param in lambda_values:
    mse_scores = []

    for train_index, test_index in kf.split(X_poly):
        X_train, X_test = X_poly[train_index], X_poly[test_index]
        y_train, y_test = y[train_index], y[test_index]

        XTX = np.dot(X_train.T, X_train)
        I = np.eye(XTX.shape[0])
        theta = np.dot(np.linalg.inv(XTX + lambda_param * I), np.dot(X_train.T, y_train))

        y_prediction_train = np.dot(X_train, theta)
        y_prediction_test = np.dot(X_test, theta)

        mse_train = np.mean((y_train - y_prediction_train) ** 2)
        mse_test = np.mean((y_test - y_prediction_test) ** 2)

        mse_scores.append({'train': mse_train, 'test': mse_test})

    # store the average mse scores for this lambda value
    mse_scores_dict[lambda_param] = {
        'train': np.mean([fold['train'] for fold in mse_scores]),
        'test': np.mean([fold['test'] for fold in mse_scores])
    }

In [ ]:
# find the lambda value with the lowest average mse on the test set
best_lambda = min(mse_scores_dict, key=lambda k: mse_scores_dict[k]['test'])
best_mse_train = mse_scores_dict[best_lambda]['train']
best_mse_test = mse_scores_dict[best_lambda]['test']

print(f'best lambda: 10^{int(np.log10(best_lambda))}')
print(f'average mse on training set: {best_mse_train}')
print(f'average mse on test set: {best_mse_test}')

### Multivariate Linear Regression using Gradient Descent

In [ ]:
X = boston.data
y = boston.target

# normalize dataset
u = np.mean(X, axis=0)
std = np.std(X, axis=0)
X = (X - u) / std

# create polynomial features of degree 2
poly = PolynomialFeatures(degree=2)
X_poly = poly.fit_transform(X)

# append the column of 1s
one = np.ones((X_poly.shape[0], 1))
X_poly = np.hstack((one, X_poly))

# initialize parameters
theta = np.zeros(X_poly.shape[1])

# hyperparameters
L = 0.01  # learning rate
epochs = 1000

# define the number of folds
k = 10
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# initialize lists to store mse values for each fold
mse_train_list = []
mse_test_list = []

# perform k-fold cross-validation
for train_index, test_index in kf.split(X_poly):
    X_train, X_test = X_poly[train_index], X_poly[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # gradient descent
    for iteration in range(epochs):
        # calculate predictions
        predictions_train = np.dot(X_train, theta)
        predictions_test = np.dot(X_test, theta)

        # calculate MSE for training set
        mse_train = np.mean((predictions_train - y_train) ** 2)

        # calculate MSE for test set
        mse_test = np.mean((predictions_test - y_test) ** 2)

        # update parameters using gradient
        gradient = np.dot(X_train.T, predictions_train - y_train) / len(y_train)
        theta -= L * gradient

    # append mse values to the lists
    mse_train_list.append(mse_train)
    mse_test_list.append(mse_test)

In [ ]:
# calculate and print the average MSE values
average_mse_train = np.mean(mse_train_list)
average_mse_test = np.mean(mse_test_list)

print(f'average mse on training set: {average_mse_train}')
print(f'average mse on test set: {average_mse_test}')

### Lasso Regression

In [166]:
X = boston.data
y = boston.target

# normalize features
X = (X - np.mean(X, axis=0)) / np.std(X, axis=0)

# add a column of ones for the intercept term
X = np.c_[np.ones(X.shape[0]), X]

# initialize parameters
theta = np.zeros(X.shape[1])

# hyperparameters
alpha = 0.01  # learning rate
lambda_param = 0.1  # regularization parameter
num_iterations = 1000

# define the number of folds
k = 10
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# initialize lists to store MSE values for each fold
mse_list = []

# perform k-fold cross-validation
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # gradient Descent
    for iteration in range(num_iterations):
        # calculate predictions
        predictions_train = np.dot(X_train, theta)

        # calculate errors
        errors = predictions_train - y_train

        # update parameters using gradient with L1 penalty
        l1_penalty = (lambda_param / len(y_train)) * np.sign(theta)
        l1_penalty[0] = 0
        gradient = (1 / len(y_train)) * np.dot(X_train.T, errors) + l1_penalty
        theta -= alpha * gradient

    # evaluate on the test set for this fold
    predictions_test = np.dot(X_test, theta)
    mse_test = np.mean((predictions_test - y_test) ** 2)

    mse_list.append(mse_test)

# calculate and print the average MSE value across all folds
average_mse = np.mean(mse_list)
print(f'average mean squared error across {k} folds: {average_mse}')

average mean squared error across 10 folds: 23.340154343671713


### Elastic Net

In [ ]:
X = boston.data
y = boston.target

# normalize features
X = (X - np.mean(X, axis=0)) / np.std(X, axis=0)

# add a column of ones for the intercept term
X = np.c_[np.ones(X.shape[0]), X]

# initialize parameters
theta = np.zeros(X.shape[1])

# hyperparameters
alpha = 0.01  # learning rate
lambda_1 = 0.1  # L1 regularization parameter
lambda_2 = 0.1  # L2 regularization parameter
num_iterations = 1000

# define the number of folds
k = 10
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# initialize lists to store MSE values for each fold
mse_list = []

# perform K-fold cross-validation
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # gradient Descent
    for iteration in range(num_iterations):
        # calculate predictions
        predictions_train = np.dot(X_train, theta)

        # calculate errors
        errors = predictions_train - y_train

        # update parameters using gradient with Elastic Net penalty
        l1_penalty = (lambda_1 / len(y_train)) * np.sign(theta)
        l2_penalty = (lambda_2 / len(y_train)) * theta
        gradient = (1 / len(y_train)) * np.dot(X_train.T, errors) + l1_penalty + l2_penalty
        theta -= alpha * gradient
        
        # calculate cost (Mean Squared Error)
        cost = np.mean((np.dot(X_train, theta) - y_train) ** 2)

    # evaluate on the test set for this fold
    predictions_test = np.dot(X_test, theta)
    mse_test = np.mean((predictions_test - y_test) ** 2)

    mse_list.append(mse_test)

# calculate and print the average MSE value across all folds
average_mse = np.mean(mse_list)
print(f'Average mean squared error across {k} folds: {average_mse}')


### Chosen model

Multivariate linear regression. The parameters are the coefficients of each feature. It is simple and easy to work with, and it generally performs well when there are linear relationships between features and the target variable. In this case, multivariate linear regression with gradient descent gave better mse scores compared to Lasso and Elastic Net. 